## rawdata  --->  NER_data

* 原来是标注的原始文件，relations,entitys,等都有
* 处理后是 source,target, 但是关系尚未标出 

* 人名  Nh
* 毒品 NDR
* 其他都是O

- O   0
- traffic_in 1
- sell_drugs_to 2
- provide_shelter_for 3
- posess          4     

In [ ]:
import json

In [ ]:
tag2id = {}
tag2id['traffic_in'] = 0
tag2id['sell_drugs_to'] = 1
tag2id['provide_shelter_for'] = 2
tag2id['posess'] = 3

In [ ]:
prepath = '../multi_data/'

filename = 'multi_train.txt'

In [ ]:
with open(prepath+filename,'r',encoding='gbk') as f:
    lines = f.readlines()
f.close()

In [ ]:
json.loads(lines[6])

In [ ]:
def update_label(start,end,sym,label_list):
    for i in range(start,end):
        if i == start:
            label_list[i] = 'B-'+sym
#             print(label_list[i])
        else:
            label_list[i] = 'I-'+sym
#     print(start,end,label_list)

In [ ]:
ner_lines = []
for line in lines:
    ner_line = {}
    con = json.loads(line)
    sentText = con['sentText']
    if len(sentText) > 510:
        continue
#         sentText = sentText[:510]
#     print(sentText)
    label = ['O']*len(sentText)
#     print(len(sentText),type(label))
    entityMentions = con['entityMentions']
    for entity in entityMentions:
        if entity['label'] == 'Nh' or entity['label'] == 'NDR':
#             print(entity['start'])
            update_label(entity['start'],entity['end'],entity['label'],label)
    ner_line['source'] = sentText
    ner_line['target'] = label
#     ner_line['relations'] = []
#     relationMentions = con['relationMentions']
#     for relamen in relationMentions:
#         entity1 = relamen['em1Text']
#         e1start = relamen['e1start']
#         entity2 = relamen['em2Text']
#         e2start = relamen['e21start']
#         label = relamen['label']
#         ner_line['relations'].append([str(e1start)+'-'+str(len(entity1),tag2id[label],str(e2start)+'-'+str(len(entity2)]))
#         ner_line['relations'].append(tag2id[label])
#         ner_line['relations'].append(str(e2start)+'-'+str(len(entity2)))
        
    ner_lines.append(ner_line)
#     print(con['entityMentions'])
#   

In [ ]:
len(lines),len(ner_lines)  ###除去了两个特长序列

In [ ]:
json.loads(line)

In [ ]:
print(ner_lines[-1])

In [ ]:
## save
with open(prepath+filename.split('.')[0]+'ner.txt','w+',encoding = 'utf-8') as f:
    for line in ner_lines:
        line_str = json.dumps(line)
        f.write(line_str+'\n')

#### 统计字数

In [ ]:
prepath = '../multi_data/'

filename = 'multi_trainner.txt'

In [ ]:

with open(prepath+filename,'r',encoding = 'utf-8') as f:
    lines = f.readlines()

In [ ]:
count_text = []
for line in lines:
    line= line.strip()
    con = json.loads(line)
    sentText = con['source']
#     print(sentText)
    count_text.append(len(sentText))

In [ ]:
from collections import Counter
print(Counter(count_text)) ## bert 句子长度设为512

## rawdata  --->  RE_data

In [1]:
import json

In [2]:
import random

In [3]:
prepath = '../multi_data/'
filename = 'multi_test.txt'

with open(prepath+filename,'r',encoding='gbk') as f:
    lines = f.readlines()
f.close()

In [4]:
len(lines)

253

In [5]:
random.seed(2020)
##  注意，因为随机数的存在每次产生的数据都可能不一样,所以利用随机种子

In [6]:
# ner_lines = []
re_lines = []
# labels = []
for line in lines:
    ner_line = {}
    con = json.loads(line)
    sentText = con['sentText']
    relationMentions = con['relationMentions']
    triples = []
    tuples = []
    entitys = []
#     e1s = []
#     e2s = []
    for relationMention in relationMentions:
        e1 = relationMention['em1Text']
        e2 = relationMention['em2Text']
        label = relationMention['label']
        triples.append((label,e1,e2))
        tuples.append((e1,e2))
        entitys.append(e1)
        entitys.append(e2)
    length = len(entitys)   
    for i in range(5):
        t1 = random.randint(0,length-1)
        e1 = entitys[t1]
        t2 = random.randint(0,length-1)
        e2 = entitys[t2]
        if (e1,e2) in tuples or e1==e2:
            continue
        if ('O',e1,e2) not in triples:
            triples.append(('O',e1,e2))
            
#     print(triples)
    
    for lee in triples:
        re_line ={}
        line_label = lee[0]
        res_line = lee[1]+'，'+lee[2]+'。'+sentText
        if len(res_line) >510:
            continue
        re_line['source'] = res_line
        re_line['target'] = line_label
        re_lines.append(re_line)
#         re_lines.append(res_line)
#         labels.append(line_label)
#         print()

        
#     break
        
    
#     entityMentions = con['entityMentions']
#     entitys = []
#     for entity in entityMentions:
#         if entity['label'] == 'Nh' or entity['label'] == 'NDR':
#             ent_pos = (entity['text'],entity['start'])
#             entitys.append(ent_pos)
# #     entitys = entitys
#     length = len(entitys)
#     for i in range(length):
#         e1,e2,label = get_label(i,length,entitys,relationMentions)
        


In [7]:
len(re_lines)

571

In [8]:
random.shuffle (re_lines)

In [9]:
len(re_lines)

571

In [10]:
re_lines[0:10]

[{'source': '冯某龄，冯某裕。阳江市江城区人民检察院指控，从2014年8月24日至9月19日，被告人冯某龄先后多次容留吸毒人员陈某存、冯某裕等人在冯某龄位于闸坡镇丹济村委会丹二社长行路兴福巷北151号的住所吸食毒品。后民警将冯某龄、冯某裕等人抓获。',
  'target': 'provide_shelter_for'},
 {'source': '甲基苯丙胺，杨某。公诉机关指控，2016年10月14日上午，被告人杨某伙同他人在简阳市简城街道×一居民房中吸食了毒品甲基苯丙胺。当日12时许，当杨某离开行至简城街道龙王庙街时，被接群众举报赶到的民警挡获，民警从杨某上衣包内检查出净重为49.39克的疑似毒品甲基苯丙胺（俗称，“冰毒”）并予以扣押。',
  'target': 'O'},
 {'source': '张某，甲基苯丙胺片剂。公诉机关指控：2015年8月21日0时许，被告人张某驾驶其车号为鄂a×××××的出租车到武汉市新洲区阳逻街东方新天地小区，以人民币200元的价格，将4颗甲基苯丙胺片剂（俗称“麻果”）卖给陈某。交易完成后，公安民警在阳逻街金融街路段将张某抓获，并于当日将其刑事拘留。经武汉市公安毒品司法鉴定中心鉴定，查获的4颗红色片剂均为毒品甲基苯丙胺，净重为0.38克，上述毒品已被公安机关收缴。公诉机关认为被告人张某具有自愿认罪的量刑情节，建议判处被告人张某拘役三个月左右，并处罚金。',
  'target': 'traffic_in'},
 {'source': '甲基苯丙胺，武某。太原市小店区人民检察院指控，2016年1月30日11时许，在太原市小店区康宁街云鼎商务楼1号楼1号商铺内，被告人武某随身携带16小包净重15.5克毒品疑似物，被公安机关依法查获。经依法鉴定，毒品疑似物中均检出甲基苯丙胺成分。',
  'target': 'O'},
 {'source': '陈某甲，陈某1。经审理查明，2014年3月至4月期间，被告人陈某1在枣阳市南关街其租住的房间内，先后三次容留陈某甲吸食毒品。2014年10月，陈某1在枣阳市大西街卫生巷的租住房间内，容留黄某吸食毒品，具体犯罪事实如下：',
  'target': 'O'},
 {'source': '李绍某，甲基苯丙胺。公诉机关指控，2014年4月1日，被告人李绍某来到佛山市南海区大沥镇三鸟市场附

In [ ]:
# from collections import Counter
# Counter(labels)

In [11]:
## save
with open(prepath+filename.split('.')[0]+'re.txt','w+',encoding = 'utf-8') as f:
    for line in re_lines:
        line_str = json.dumps(line)
        f.write(line_str+'\n')